In [4]:
import newspaper
import feedparser


def scrape_news_from_feed(feed_url):
    articles = []
    feed = feedparser.parse(feed_url)
    for entry in feed.entries:
        # create a newspaper article object
        article = newspaper.Article(entry.link)
        # download and parse the article
        article.download()
        article.parse()
        # extract relevant information
        articles.append({
            'title': article.title,
            'author': article.authors,
            'publish_date': article.publish_date,
            'content': article.text
        })
    return articles


feed_url = 'https://vnexpress.net/ong-trump-lam-viec-voi-nga-de-hon-ukraine-4858276.html'
articles = scrape_news_from_feed(feed_url)

# print the extracted articles
for article in articles:
    print('Title:', article['title'])
    print('Author:', article['author'])
    print('Publish Date:', article['publish_date'])
    print('Content:', article['content'])
    print()


In [5]:
print(articles)

[]


## Using LlamaIndex

In [1]:
from llama_index.readers.web import SimpleWebPageReader

In [2]:
documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["https://vnexpress.net/tai-sao-phu-nu-doi-binh-dang-nhung-bat-dan-ong-lam-tru-cot-4857061.html"]
)

In [6]:
print(documents[0].text_resource.text)

[](javascript:;) [ ![VnExpress - Bao tieng Viet nhieu nguoi xem
nhat](https://s1.vnecdn.net/vnexpress/restruct/i/v9559/v2_2019/pc/graphics/logo.svg)
](/ "Báo VnExpress - Báo tiếng Việt nhiều người xem nhất") Thứ bảy, 8/3/2025

[ Mới nhất ](/tin-tuc-24h "Mới nhất")

Tin theo khu vực

  * [Hà Nội](https://vnexpress.net/topic/ha-noi-26482 "Hà Nội")
  * [TP Hồ Chí Minh](https://vnexpress.net/topic/tp-ho-chi-minh-26483 "TP Hồ Chí Minh")

[International](https://e.vnexpress.net/ "VnExpress International") [
![](https://s1.vnecdn.net/vnexpress/restruct/c/v3203/v2_2019/pc/images/graphics/menu-
myvne.svg) ](/myvne)

  * [ ](/ "Trang chủ")
  * [Mới nhất](/tin-tuc-24h "Mới nhất")
  * [ Thời sự ](/thoi-su "Thời sự")
  * [ Thế giới ](/the-gioi "Thế giới")
  * [ Kinh doanh ](/kinh-doanh "Kinh doanh")
  * [ Công nghệ ](/cong-nghe "Công nghệ")
  * [ Khoa học ](/khoa-hoc "Khoa học")
  * [ Video ](https://video.vnexpress.net "Video")
  * [ Podcasts ](/podcast "Podcasts")
  * [ Góc nhìn ](/goc-nhin "Góc 

## Test 

In [7]:
import requests
from bs4 import BeautifulSoup
import html2text
import logging
from urllib.parse import urljoin, urlparse

def web_to_markdown(url, output_file=None, include_images=True, include_links=True):
    """
    Convert the contents of a web page to Markdown format.
    
    Args:
        url (str): The URL of the web page to convert.
        output_file (str, optional): Path to save the Markdown output. If None, returns the markdown as string.
        include_images (bool): Whether to include image references in the output.
        include_links (bool): Whether to include hyperlinks in the output.
    
    Returns:
        str: The Markdown content if output_file is None, otherwise None.
    """
    try:
        # Set up logging
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        
        # Validate URL
        if not url.startswith(('http://', 'https://')):
            url = 'https://' + url
        
        # Make request with a realistic user agent
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        logging.info(f"Fetching content from {url}")
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        # Parse HTML with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Remove script and style elements
        for script in soup(["script", "style"]):
            script.decompose()
        
        # Configure html2text
        converter = html2text.HTML2Text()
        converter.ignore_links = not include_links
        converter.ignore_images = not include_images
        converter.body_width = 0  # Don't wrap text
        converter.protect_links = True  # Don't convert links to references
        
        # Fix relative URLs if needed
        if include_links or include_images:
            base_url = "{0.scheme}://{0.netloc}".format(urlparse(url))
            
            # Fix relative links
            if include_links:
                for a_tag in soup.find_all('a', href=True):
                    if not a_tag['href'].startswith(('http://', 'https://', 'mailto:', 'tel:')):
                        a_tag['href'] = urljoin(base_url, a_tag['href'])
            
            # Fix relative image sources
            if include_images:
                for img_tag in soup.find_all('img', src=True):
                    if not img_tag['src'].startswith(('http://', 'https://', 'data:')):
                        img_tag['src'] = urljoin(base_url, img_tag['src'])
        
        # Convert to markdown
        markdown = converter.handle(str(soup))
        
        # Clean up the markdown
        markdown = markdown.replace('\n\n\n\n', '\n\n')
        
        # Save to file or return as string
        if output_file:
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(markdown)
            logging.info(f"Markdown saved to {output_file}")
            return None
        else:
            return markdown
            
    except requests.exceptions.RequestException as e:
        logging.error(f"Request error: {e}")
        raise
    except Exception as e:
        logging.error(f"Conversion error: {e}")
        raise

# Example usage
if __name__ == "__main__":
    url = "https://vnexpress.net/tai-sao-phu-nu-doi-binh-dang-nhung-bat-dan-ong-lam-tru-cot-4857061.html"
    markdown = web_to_markdown(url)
    print(markdown)

2025-03-08 07:49:53,124 - INFO - Fetching content from https://vnexpress.net/tai-sao-phu-nu-doi-binh-dang-nhung-bat-dan-ong-lam-tru-cot-4857061.html


[](<javascript:;>) [ ![VnExpress - Bao tieng Viet nhieu nguoi xem nhat](https://s1.vnecdn.net/vnexpress/restruct/i/v9559/v2_2019/pc/graphics/logo.svg) ](<https://vnexpress.net/> "Báo VnExpress - Báo tiếng Việt nhiều người xem nhất") Thứ bảy, 8/3/2025

[ Mới nhất ](<https://vnexpress.net/tin-tuc-24h> "Mới nhất")

Tin theo khu vực 

  * [Hà Nội](<https://vnexpress.net/topic/ha-noi-26482> "Hà Nội")
  * [TP Hồ Chí Minh](<https://vnexpress.net/topic/tp-ho-chi-minh-26483> "TP Hồ Chí Minh")

[International](<https://e.vnexpress.net/> "VnExpress International") [ ![](https://s1.vnecdn.net/vnexpress/restruct/c/v3203/v2_2019/pc/images/graphics/menu-myvne.svg) ](<https://vnexpress.net/myvne>)

  * [ ](<https://vnexpress.net/> "Trang chủ")
  * [Mới nhất](<https://vnexpress.net/tin-tuc-24h> "Mới nhất")
  * [ Thời sự ](<https://vnexpress.net/thoi-su> "Thời sự")
  * [ Thế giới ](<https://vnexpress.net/the-gioi> "Thế giới")
  * [ Kinh doanh ](<https://vnexpress.net/kinh-doanh> "Kinh doanh")
  * [ Công

## Adding post processing 

In [8]:
import re

start_at_first_h1 = True

# Clean up the markdown
markdown = markdown.replace('\n\n\n\n', '\n\n')

# Post-processing: Remove all content before the first H1 header if requested
if start_at_first_h1:
    logging.info("Removing content before first H1 header")
    # Look for the first proper Markdown H1 header (# at the start of a line)
    h1_match = re.search(r'^\s*# .+', markdown, re.MULTILINE)
    if h1_match:
        # Keep only the content starting from the first H1 header
        markdown = markdown[h1_match.start():]
        logging.info("Content before first H1 header removed")
    else:
        logging.warning("No H1 header found in the content")

print(markdown)
        

2025-03-08 07:54:37,290 - INFO - Removing content before first H1 header
2025-03-08 07:54:37,291 - INFO - Content before first H1 header removed



#  Tại sao phụ nữ đòi bình đẳng nhưng bắt đàn ông làm trụ cột? 

"Quy tắc số 1" của Minh An khi hẹn hò là đàn ông phải thanh toán ít nhất năm buổi đầu tiên, từ buổi thứ 6 cô có thể chia sẻ một phần nhỏ.

"Làm gì có bình đẳng thực sự. Phụ nữ vẫn chịu nhiều bất công, đàn ông chi nhiều hơn mới công bằng", cô gái ở TP HCM lập luận.

Cô cho rằng nam giới có thu nhập trung bình cao hơn, trong khi phụ nữ phải đầu tư nhiều hơn vào ngoại hình, từ trang điểm, váy áo đến làm tóc, để đáp ứng kỳ vọng xã hội. Việc đàn ông trả tiền cũng là cách thể hiện sự nghiêm túc trong mối quan hệ. "Tiền ở đâu, đàn ông để tâm ở đó", cô kết luận.

Tuy nhiên, nhiều nam giới không đồng tình, cho rằng chính tư duy này tạo ra bất bình đẳng. Tuấn Dũng, nhân viên chứng khoán 31 tuổi ở Hà Nội, cho biết luôn thanh toán các buổi hẹn, thường rơi vào khoảng 500.000 đồng mỗi lần. Nhưng với anh, hành động này đơn thuần là phép lịch sự, không phải trách nhiệm của nam giới.

Điều khiến Dũng bối rối là những tiêu chuẩn mâu thuẫn

## Download paper

In [2]:
import wget
url = 'https://arxiv.org/pdf/2503.04625'


file_Path = 'research_Paper_3.pdf'
wget.download(url, file_Path)
print('downloaded')

downloaded
